# Quantum Die

In [1]:
import math
from pyquil import Program, get_qc
from pyquil.gates import H

In [2]:
def qubits_needed(number_of_sides):
    """
    The number of qubits needed for a die of n faces.
    """
    return int(math.ceil(math.log(number_of_sides, 2)))


def get_qvm(number_of_sides):
    """
    Get a QVM to simulate the requested number of sides.
    """
    return get_qc(f"{qubits_needed(number_of_sides)}q-qvm")


def die_program(number_of_sides):
    """
    Generate a quantum program to roll a die of n faces.
    """
    prog = Program()
    n_qubits = qubits_needed(number_of_sides)
    ro = prog.declare("ro", "BIT", n_qubits)
    # Hadamard initialize.
    for q in range(n_qubits):
        prog.inst(H(q))
    # Measure everything.
    for q in range(n_qubits):
        prog.measure(q, ro[q])
    return prog


def process_results(results):
    """
    Convert n digit binary result from the QVM to a value on a die.
    """
    raw_results = results[0]
    processing_result = 0
    for each_qubit_measurement in raw_results:
        processing_result = 2 * processing_result + each_qubit_measurement
    # Convert from 0 indexed to 1 indexed
    die_value = processing_result + 1
    return die_value


def roll_die(qvm, number_of_sides):
    """
    Roll an n-sided quantum die.
    """
    die_compiled = qvm.compile(die_program(number_of_sides))
    return process_results(qvm.run(die_compiled))

In [3]:
number_of_sides = 5

qvm = get_qvm(number_of_sides)
die_value = roll_die(qvm, number_of_sides)
while die_value > number_of_sides:
    die_value = roll_die(qvm, number_of_sides)
print(f"The result is: {die_value}")

The result is: 2
